1: DimDate Table

In [1]:
%pip install --upgrade sqlalchemy
%pip install --upgrade pyodbc

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\sfsan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\sfsan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import pyodbc
print(pyodbc.drivers())

['SQL Server', 'SQL Server Native Client RDA 11.0', 'ODBC Driver 17 for SQL Server']


In [3]:
import re
import pandas as pd
from sqlalchemy import create_engine

In [4]:
# Gegevens voor de verbinding
server = r"DESKTOP-1DRSSG0\SQLEXPRESS"  # Servernaam of IP-adres van je SQL Server
database = "DEP1_DWH"  # Naam van je database

# Maak de verbindingsstring met Windows Authenticatie (Integrated Security)
engine = create_engine("mssql+pyodbc://@{}/{}?driver=ODBC+Driver+17+for+SQL+Server".format(server, database))

In [5]:
# Maak een range van datums
date_list = pd.date_range(start="01-01-2010", end="31-12-2025", freq='D')

months_translation = {
    'January': 'Januari', 'February': 'Februari', 'March': 'Maart', 'April': 'April',
    'May': 'Mei', 'June': 'Juni', 'July': 'Juli', 'August': 'Augustus', 
    'September': 'September', 'October': 'Oktober', 'November': 'November', 'December': 'December'
}

days_translation = {
    'Monday': 'Maandag', 'Tuesday': 'Dinsdag', 'Wednesday': 'Woensdag', 'Thursday': 'Donderdag',
    'Friday': 'Vrijdag', 'Saturday': 'Zaterdag', 'Sunday': 'Zondag'
}

dim_date_df = pd.DataFrame({
    'DateKey': date_list.strftime('%Y%m%d').astype(int),  # YYYYMMDD als key
    'FullDate': date_list.date,  # Volledige datum
    'MonthNameDutch': date_list.strftime('%B').map(months_translation),  # Maandnaam (kan vertaald worden)
    'MonthNameEN': date_list.strftime('%B'),  # Maandnaam in Engels
    'DayNameDutch': date_list.strftime('%A').map(days_translation),  # Dagnaam in Nederlands
    'DayNameEN': date_list.strftime('%A'),  # Dagnaam in Engels
    'QuarterName': 'Q' + date_list.quarter.astype(str),  # Kwartaal als 'Q1', 'Q2', ...
    'QuarterNumber': date_list.quarter  # Kwartaalnummer (1-4)
})

In [ ]:
# Schrijf naar SQL Server
dim_date_df.to_sql('DimDate', con=engine, if_exists='append', index=False)

In [6]:
dim_date_df.head()

,DateKey,FullDate,MonthNameDutch,MonthNameEN,DayNameDutch,DayNameEN,QuarterName,QuarterNumber
0,20100101,2010-01-01,Januari,January,Vrijdag,Friday,Q1,1
1,20100102,2010-01-02,Januari,January,Zaterdag,Saturday,Q1,1
2,20100103,2010-01-03,Januari,January,Zondag,Sunday,Q1,1
3,20100104,2010-01-04,Januari,January,Maandag,Monday,Q1,1
4,20100105,2010-01-05,Januari,January,Dinsdag,Tuesday,Q1,1


2: DimTime Table

In [7]:
def generate_dim_time():
    time_data = []

    for hour in range(0, 24):
        for minute in range(0, 60):
            am_pm = 'AM' if hour < 12 else 'PM'
            hour_12 = hour if 1 <= hour <= 12 else (12 if hour == 0 or hour == 24 else hour - 12)
            time_key = f"{hour:02}{minute:02}"
            full_time = f"{hour:02}:{minute:02}:00"
            
            time_data.append({
                "TimeKey": time_key,
                "Hour": hour_12,
                "Minutes": minute,
                "FullTime": full_time,
                "TimeAM_PM": am_pm
            })
    
    return pd.DataFrame(time_data)

# Data genereren
dim_time_df = generate_dim_time()

In [ ]:
# Data naar SQL Server schrijven
dim_time_df.to_sql("DimTime", con=engine, if_exists="append", index=False)

In [8]:
dim_time_df.head()

,TimeKey,Hour,Minutes,FullTime,TimeAM_PM
0,0000,12,0,00:00:00,AM
1,0001,12,1,00:01:00,AM
2,0002,12,2,00:02:00,AM
3,0003,12,3,00:03:00,AM
4,0004,12,4,00:04:00,AM


3: DimWeatherStation Table

In [9]:
# Lees de CSV voor weerstations
weather_station_df = pd.read_csv('../data/input/aws_station.csv')

# Verwerk de kolommen
weather_station_df.rename(columns={
    "code": "WeatherStationID",
    "name": "WeatherStationName",
    "altitude": "Altitude",
    "the_geom": "Coordinates"
}, inplace=True)



# Functie om Latitude en Longitude te extraheren uit 'the_geom' kolom
def extract_lat_lon(geom):
    match = re.search(r"POINT \(([\d\.-]+) ([\d\.-]+)\)", geom)
    if match:
        lon, lat = match.groups()
        return float(lat), float(lon)
    return None, None

# Latitude en Longitude kolommen toevoegen
weather_station_df["Latitude"], weather_station_df["Longitude"] = zip(*weather_station_df["Coordinates"].apply(extract_lat_lon))

# Onnodige kolom verwijderen
weather_station_df.drop(columns=["Coordinates"], inplace=True)
weather_station_df = weather_station_df.drop(['FID', 'date_begin', 'date_end'], axis = 1)

In [6]:
# Data naar SQL Server schrijven
weather_station_df.to_sql("DimWeatherStation", con=engine, if_exists="append", index=False)

8

In [10]:


# Add a unique key column (starting from 1)
weather_station_df.insert(0, 'WeatherStationKey', range(1, len(weather_station_df) + 1))



In [11]:
weather_station_df.head()

,WeatherStationKey,WeatherStationID,WeatherStationName,Altitude,Latitude,Longitude
0,1,6472,HUMAIN,295.3,5.255,50.194
1,2,6438,STABROEK,4.0,4.364,51.325
2,3,6464,RETIE,21.5,5.027,51.221
3,4,6447,UCCLE,100.6,4.358,50.797
4,5,6434,MELLE,15.0,3.816,50.980


4: FactWeather Table

In [12]:
# Load the CSV file
file_path = "../data/input/aws_1day.csv"

weather_data_df = pd.read_csv(file_path)

weather_data_df.head(), weather_data_df.columns

(                                       FID              the_geom  code  \
 0  aws_1day.fid--55b99fcb_194dc70ea36_71e2  POINT (51.348 3.202)  6418   
 1  aws_1day.fid--55b99fcb_194dc70ea36_71e3  POINT (51.075 4.525)  6439   
 2  aws_1day.fid--55b99fcb_194dc70ea36_71e4  POINT (51.325 4.364)  6438   
 3  aws_1day.fid--55b99fcb_194dc70ea36_71e5   POINT (50.916 5.45)  6477   
 4  aws_1day.fid--55b99fcb_194dc70ea36_71e6  POINT (50.797 4.358)  6447   
 
              timestamp  precip_quantity  temp_avg  temp_max  temp_min  \
 0  2020-10-14T00:00:00             0.31     11.29     12.80     10.00   
 1  2020-10-14T00:00:00             0.00      8.94     12.23      5.15   
 2  2020-10-14T00:00:00             0.00      9.06     12.33      5.91   
 3  2020-10-15T00:00:00             0.00      9.40     11.16      7.92   
 4  2020-10-16T00:00:00             0.00      9.01     10.72      5.62   
 
    temp_grass_pt100_avg  temp_soil_avg  ...  temp_soil_avg_50cm  \
 0                   NaN          

In [13]:
weather_data_df = weather_data_df.drop(['FID', 'the_geom', 'qc_flags'], axis = 1)
weather_data_df.head()

,code,timestamp,precip_quantity,temp_avg,temp_max,temp_min,temp_grass_pt100_avg,temp_soil_avg,temp_soil_avg_5cm,temp_soil_avg_10cm,temp_soil_avg_20cm,temp_soil_avg_50cm,wind_speed_10m,wind_speed_avg_30m,wind_gusts_speed,humidity_rel_shelter_avg,pressure,sun_duration,short_wave_from_sky_avg,sun_int_avg
0,6418,2020-10-14T00:00:00,0.31,11.29,12.80,10.00,NaN,NaN,NaN,NaN,NaN,NaN,7.74,NaN,13.79,75.37,1017.57,82.33,NaN,25.30
1,6439,2020-10-14T00:00:00,0.00,8.94,12.23,5.15,9.18,8.90,9.41,9.99,11.20,13.21,2.60,NaN,7.16,86.44,1017.32,11.75,61.83,10.52
2,6438,2020-10-14T00:00:00,0.00,9.06,12.33,5.91,8.93,8.88,9.32,9.61,9.84,12.83,3.00,NaN,8.96,86.75,1018.37,33.08,63.74,19.22
3,6477,2020-10-15T00:00:00,0.00,9.40,11.16,7.92,9.70,9.33,9.32,9.54,9.10,11.53,3.53,NaN,8.73,82.62,1015.69,15.67,56.01,10.12
4,6447,2020-10-16T00:00:00,0.00,9.01,10.72,5.62,9.88,9.42,9.64,10.05,10.65,12.23,NaN,2.67,6.87,82.88,1010.36,NaN,51.23,NaN


In [14]:
weather_data_df = weather_data_df.merge(weather_station_df, how='inner', left_on="code", right_on='WeatherStationID')
weather_data_df.head()

,code,timestamp,precip_quantity,temp_avg,temp_max,temp_min,temp_grass_pt100_avg,temp_soil_avg,temp_soil_avg_5cm,temp_soil_avg_10cm,...,pressure,sun_duration,short_wave_from_sky_avg,sun_int_avg,WeatherStationKey,WeatherStationID,WeatherStationName,Altitude,Latitude,Longitude
0,6418,2020-10-14T00:00:00,0.31,11.29,12.80,10.00,NaN,NaN,NaN,NaN,...,1017.57,82.33,NaN,25.30,8,6418,ZEEBRUGGE,12.1,3.202,51.348
1,6439,2020-10-14T00:00:00,0.00,8.94,12.23,5.15,9.18,8.90,9.41,9.99,...,1017.32,11.75,61.83,10.52,7,6439,SINT-KATELIJNE-WAVER,8.3,4.525,51.075
2,6438,2020-10-14T00:00:00,0.00,9.06,12.33,5.91,8.93,8.88,9.32,9.61,...,1018.37,33.08,63.74,19.22,2,6438,STABROEK,4.0,4.364,51.325
3,6447,2020-10-16T00:00:00,0.00,9.01,10.72,5.62,9.88,9.42,9.64,10.05,...,1010.36,NaN,51.23,NaN,4,6447,UCCLE,100.6,4.358,50.797
4,6464,2020-10-17T00:00:00,0.00,7.97,13.09,4.00,8.66,9.96,10.23,10.32,...,1019.12,219.67,93.01,89.48,3,6464,RETIE,21.5,5.027,51.221


In [15]:
weather_data_df = weather_data_df.drop(['WeatherStationName', 'Latitude', 'Longitude', 'Altitude', 'WeatherStationID'], axis = 1)
weather_data_df.head()

,code,timestamp,precip_quantity,temp_avg,temp_max,temp_min,temp_grass_pt100_avg,temp_soil_avg,temp_soil_avg_5cm,temp_soil_avg_10cm,...,temp_soil_avg_50cm,wind_speed_10m,wind_speed_avg_30m,wind_gusts_speed,humidity_rel_shelter_avg,pressure,sun_duration,short_wave_from_sky_avg,sun_int_avg,WeatherStationKey
0,6418,2020-10-14T00:00:00,0.31,11.29,12.80,10.00,NaN,NaN,NaN,NaN,...,NaN,7.74,NaN,13.79,75.37,1017.57,82.33,NaN,25.30,8
1,6439,2020-10-14T00:00:00,0.00,8.94,12.23,5.15,9.18,8.90,9.41,9.99,...,13.21,2.60,NaN,7.16,86.44,1017.32,11.75,61.83,10.52,7
2,6438,2020-10-14T00:00:00,0.00,9.06,12.33,5.91,8.93,8.88,9.32,9.61,...,12.83,3.00,NaN,8.96,86.75,1018.37,33.08,63.74,19.22,2
3,6447,2020-10-16T00:00:00,0.00,9.01,10.72,5.62,9.88,9.42,9.64,10.05,...,12.23,NaN,2.67,6.87,82.88,1010.36,NaN,51.23,NaN,4
4,6464,2020-10-17T00:00:00,0.00,7.97,13.09,4.00,8.66,9.96,10.23,10.32,...,12.35,0.46,NaN,2.97,87.16,1019.12,219.67,93.01,89.48,3


In [16]:
# We converteren timestamp naar DateKey.
weather_data_df['DateKey'] = weather_data_df['timestamp'].str[0:4] + weather_data_df['timestamp'].str[5:7] + weather_data_df['timestamp'].str[8:10]
# DimTime toevoegen: eerst veld toevoegen, dan de waarden mergen.
weather_data_df['Time'] = weather_data_df['timestamp'].str[-8:]
weather_data_df = weather_data_df.merge(dim_time_df, how='inner', left_on="Time", right_on='FullTime')

# weather_data_df = weather_data_df.drop(['timestamp', 'Hour', 'Minutes', 'FullTime', 'TimeAM_PM'], axis = 1)

weather_data_df.head()

,code,timestamp,precip_quantity,temp_avg,temp_max,temp_min,temp_grass_pt100_avg,temp_soil_avg,temp_soil_avg_5cm,temp_soil_avg_10cm,...,short_wave_from_sky_avg,sun_int_avg,WeatherStationKey,DateKey,Time,TimeKey,Hour,Minutes,FullTime,TimeAM_PM
0,6418,2020-10-14T00:00:00,0.31,11.29,12.80,10.00,NaN,NaN,NaN,NaN,...,NaN,25.30,8,20201014,00:00:00,0000,12,0,00:00:00,AM
1,6439,2020-10-14T00:00:00,0.00,8.94,12.23,5.15,9.18,8.90,9.41,9.99,...,61.83,10.52,7,20201014,00:00:00,0000,12,0,00:00:00,AM
2,6438,2020-10-14T00:00:00,0.00,9.06,12.33,5.91,8.93,8.88,9.32,9.61,...,63.74,19.22,2,20201014,00:00:00,0000,12,0,00:00:00,AM
3,6447,2020-10-16T00:00:00,0.00,9.01,10.72,5.62,9.88,9.42,9.64,10.05,...,51.23,NaN,4,20201016,00:00:00,0000,12,0,00:00:00,AM
4,6464,2020-10-17T00:00:00,0.00,7.97,13.09,4.00,8.66,9.96,10.23,10.32,...,93.01,89.48,3,20201017,00:00:00,0000,12,0,00:00:00,AM


In [17]:
# we hernoemen de kolommen volgens de namen die in de DWH zitten. 
weather_data_df = weather_data_df.rename(columns={"precip_quantity": "PrecipQuantity","temp_avg": "TempAvg","temp_max": "TempMax","temp_min": "TempMin",
                                                  "temp_grass_pt100_avg": "TempGrassPt100Avg","temp_soil_avg": "TempSoilAvg","temp_soil_avg_5cm": "TempSoilAvg5cm",
                                                  "temp_soil_avg_10cm": "TempSoilAvg10cm","temp_soil_avg_20cm": "TempSoilAvg20cm",
                                                  "temp_soil_avg_50cm": "TempSoilAvg50cm","wind_speed_10m": "WindSpeed10m",
                                                  "wind_speed_avg_30m": "WindSpeedAvg30m","wind_gusts_speed": "WindGustsSpeed",
                                                  "humidity_rel_shelter_avg": "HumidityRelShelterAvg","pressure": "Pressure","sun_duration": "SunDuration",
                                                  "short_wave_from_sky_avg": "ShortWaveFromSkyAvg","sun_int_avg": "SunIntAvg"})

weather_data_df = weather_data_df.reindex(columns=["DateKey", "TimeKey", "WeatherStationKey", "PrecipQuantity", "TempAvg", "TempMax", "TempMin",
                                                    "TempGrassPt100Avg", "TempSoilAvg", "TempSoilAvg5cm", "TempSoilAvg10cm", 
                                                    "TempSoilAvg20cm", "TempSoilAvg50cm", "WindSpeed10m", "WindSpeedAvg30m", 
                                                    "WindGustsSpeed", "HumidityRelShelterAvg", "Pressure", "SunDuration", "ShortWaveFromSkyAvg", 
                                                    "SunIntAvg"])

weather_data_df.head()

,DateKey,TimeKey,WeatherStationKey,PrecipQuantity,TempAvg,TempMax,TempMin,TempGrassPt100Avg,TempSoilAvg,TempSoilAvg5cm,...,TempSoilAvg20cm,TempSoilAvg50cm,WindSpeed10m,WindSpeedAvg30m,WindGustsSpeed,HumidityRelShelterAvg,Pressure,SunDuration,ShortWaveFromSkyAvg,SunIntAvg
0,20201014,0000,8,0.31,11.29,12.80,10.00,NaN,NaN,NaN,...,NaN,NaN,7.74,NaN,13.79,75.37,1017.57,82.33,NaN,25.30
1,20201014,0000,7,0.00,8.94,12.23,5.15,9.18,8.90,9.41,...,11.20,13.21,2.60,NaN,7.16,86.44,1017.32,11.75,61.83,10.52
2,20201014,0000,2,0.00,9.06,12.33,5.91,8.93,8.88,9.32,...,9.84,12.83,3.00,NaN,8.96,86.75,1018.37,33.08,63.74,19.22
3,20201016,0000,4,0.00,9.01,10.72,5.62,9.88,9.42,9.64,...,10.65,12.23,NaN,2.67,6.87,82.88,1010.36,NaN,51.23,NaN
4,20201017,0000,3,0.00,7.97,13.09,4.00,8.66,9.96,10.23,...,10.82,12.35,0.46,NaN,2.97,87.16,1019.12,219.67,93.01,89.48


In [ ]:
# Data naar SQL Server schrijven
weather_data_df.to_sql("FactWeather", con=engine, if_exists="append", index=False)

5: FactBelpex Table

In [ ]:
from datetime import datetime

# Read the CSV with proper headers and separator
belpex_df = pd.read_csv('../data/input/BelpexFilter.csv', encoding='Windows-1252', sep=';', names=['Date', 'BelpexPrice'], skiprows=1)

# Clean and convert 'BelpexPrice'
belpex_df['BelpexPrice'] = belpex_df['BelpexPrice'].str.replace('€', '', regex=False).str.replace(',', '.', regex=False).str.strip()
belpex_df['BelpexPrice'] = pd.to_numeric(belpex_df['BelpexPrice'], errors='coerce')

# Parse 'Date' into DateKey and FullTime using datetime
belpex_df['Date'] = belpex_df['Date'].str.strip()  # Remove extra spaces
belpex_df['DateKey'] = belpex_df['Date'].apply(lambda x: datetime.strptime(x.split()[0], '%d/%m/%Y').strftime('%Y%m%d'))
belpex_df['FullTime'] = belpex_df['Date'].apply(lambda x: x.split()[1])  # Extract time part

# Merge with DimTime to get TimeKey
belpex_df = belpex_df.merge(dim_time_df[['FullTime', 'TimeKey']], how='inner', on='FullTime')

# Drop unnecessary columns
belpex_df = belpex_df.drop(['FullTime', 'Date'], axis=1)

# Write to SQL Server
belpex_df.to_sql('FactBelpex', con=engine, if_exists='append', index=False)

##### 6: FactNetworkCosts

In [19]:
from datetime import datetime


csv_file_path = "../data/input/Distributiekosten.csv"
df_distributiekosten = pd.read_csv(csv_file_path)

# Datum omzetten naar DateKey (YYYYMMDD)
df_distributiekosten["DateKey"] = df_distributiekosten["Van"].apply(lambda x: datetime.strptime(x, "%d/%m/%Y").strftime("%Y%m%d"))

df_distributiekosten.head()


,Van,Tot,Intercommunale,Capaciteitstarief_Digitale_meter,Afnametarief_Digitale_meter_Normaal,Afnametarief_Digitale_meter_Exclusief_nacht,Capaciteitstarief_Klassieke_meter,Afnametarief_Klassieke_meter_Normaal,Afnametarief_Klassieke_meter_Exclusief_nacht,Prosumententarief,Tarief_databeheer_Jaar_en_maandgelezen_meters,Tarief_databeheer_Kwartiergelezen_meters,DateKey
0,01/01/2023,31/12/2023,Fluvius Antwerpen,40.0309,3.74193,2.60192,100.0746,5.70831,4.56830,38.56,13.39,14.53,20230101
1,01/01/2023,31/12/2023,Fluvius Limburg,37.6491,3.94475,2.81516,94.1280,6.08459,4.95500,41.11,13.39,14.53,20230101
2,01/01/2023,31/12/2023,Fluvius West,41.0824,3.81831,2.72467,102.7034,6.10955,5.01591,41.28,13.39,14.53,20230101
3,01/01/2023,31/12/2023,Gaselwest,48.7615,4.98204,3.36755,121.9000,7.40908,5.79459,50.05,13.39,14.53,20230101
4,01/01/2023,31/12/2023,Imewo,43.5071,4.01029,2.82576,108.7666,6.23503,5.05050,42.12,13.39,14.53,20230101


In [ ]:
# Kolommen hernoemen
column_mapping = {
    "Capaciteitstarief_Digitale_meter": "CapacityTariff_DigitalMeter",
    "Afnametarief_Digitale_meter_Normaal": "ConsumptionTariff_DigitalMeter_Normal",
    "Afnametarief_Digitale_meter_Exclusief_nacht": "ConsumptionTariff_DigitalMeter_ExclusiveNight",
    "Capaciteitstarief_Klassieke_meter": "CapacityTariff_ClassicMeter",
    "Afnametarief_Klassieke_meter_Normaal": "ConsumptionTariff_ClassicMeter_Normal",
    "Afnametarief_Klassieke_meter_Exclusief_nacht": "ConsumptionTariff_ClassicMeter_ExclusiveNight",
    "Prosumententarief": "ProsumerTariff",
    "Tarief_databeheer_Jaar_en_maandgelezen_meters": "DataManagementTariff_YearlyMonthlyReadMeters",
    "Tarief_databeheer_Kwartiergelezen_meters": "DataManagementTariff_QuarterlyReadMeters",
    "Intercommunale": "NetworkOperator"
}
df_distributiekosten = df_distributiekosten.rename(columns=column_mapping)

df_distributiekosten.head()

In [ ]:
columns = [
    "DateKey", "NetworkOperator", "CapacityTariff_DigitalMeter",
    "ConsumptionTariff_DigitalMeter_Normal", "ConsumptionTariff_DigitalMeter_ExclusiveNight",
    "CapacityTariff_ClassicMeter", "ConsumptionTariff_ClassicMeter_Normal",
    "ConsumptionTariff_ClassicMeter_ExclusiveNight", "ProsumerTariff",
    "DataManagementTariff_YearlyMonthlyReadMeters", "DataManagementTariff_QuarterlyReadMeters"
]
df_distributiekosten = df_distributiekosten[columns]

df_distributiekosten.to_sql('FactNetworkCosts', con=engine, if_exists='append', index=False)



6: Verbruikersdata



In [ ]:
# dowmload de dataset: https://opendata.fluvius.be/explore/dataset/1_50-verbruiksprofielen-dm-elek-kwartierwaarden-voor-een-volledig-jaar/information/
# Lees de CSV met alle verbruikersdata in een pandas dataframe.
csv_file_path = "../data/input/P6269_1_50_DMK_Sample_Elek.csv"
df_verbruikersdata = pd.read_csv(csv_file_path, sep=';', encoding='utf-8')

In [ ]:
# Hernoem kolom om te matchen met tabel in sql server
df_verbruikersdata.rename(columns={
    'PV-Installatie_Indicator': 'PV_Installatie_Indicator',
}, inplace=True)

In [ ]:

df_verbruikersdata.to_sql('DimUser_Staging', con=engine, if_exists='append', index=False, chunksize=5000)

# Laatste stap, voer sql script uit: ../DWH/Staging_To_DimUser.sql

7: Overige Tabellen

- Voor de overige tabellen, volg dezelfde logica:
    - Lees de CSV’s.
    - Voeg de benodigde foreign keys toe.
    - Schrijf de data weg naar de juiste tabellen via bulk-insert of andere batch methoden.

Algemeen:
Voor alle bulk-insert taken moet je zorgen voor een efficiënte schrijfmethode naar SQL Server, bijvoorbeeld:

- to_sql() in combinatie met een SQLAlchemy engine.
- Bulk-insert via pyodbc of tools zoals bcp.
- Gebruik maken van BULK INSERT in SQL Server voor het snel inladen van grote datasets.

Voeg contract types toe:

In [ ]:
# Create a list of energy contracts with their types
energy_contracts = [
    # BOLT
    {'Provider': 'BOLT', 'ContractName': 'ELEKTRICITEIT', 'ContractType': 'Variabel'},
    {'Provider': 'BOLT', 'ContractName': 'VAST', 'ContractType': 'Vast'},
    
    # DATS24
    {'Provider': 'DATS24', 'ContractName': 'ELEKTRICITEIT', 'ContractType': 'Variabel'},
    
    # ENECO
    {'Provider': 'ENECO', 'ContractName': 'ZON WIND FLEX', 'ContractType': 'Variabel'},
    
    # ENERGIE-BE
    {'Provider': 'ENERGIE-BE', 'ContractName': 'VARIABEL', 'ContractType': 'Variabel'},
    {'Provider': 'ENERGIE-BE', 'ContractName': 'VAST', 'ContractType': 'Vast'},
    
    # LUMINUS
    {'Provider': 'LUMINUS', 'ContractName': 'DYNAMIC', 'ContractType': 'Dynamisch'},
    
    # OCTA+
    {'Provider': 'OCTA+', 'ContractName': 'DYNAMIC', 'ContractType': 'Dynamisch'},
    {'Provider': 'OCTA+', 'ContractName': 'ECO CLEAR', 'ContractType': 'Variabel'},
    {'Provider': 'OCTA+', 'ContractName': 'FIXED', 'ContractType': 'Vast'},
    {'Provider': 'OCTA+', 'ContractName': 'SMART VARIABEL', 'ContractType': 'Variabel'},
    
    # TOTALENERGIES
    {'Provider': 'TOTALENERGIES', 'ContractName': 'PIXEL', 'ContractType': 'Variabel'},
    {'Provider': 'TOTALENERGIES', 'ContractName': 'PIXEL NEXT VAST', 'ContractType': 'Vast'},
    {'Provider': 'TOTALENERGIES', 'ContractName': 'PIXEL EDRIVE', 'ContractType': 'Variabel'},
    {'Provider': 'TOTALENERGIES', 'ContractName': 'PIXIE', 'ContractType': 'Variabel'}
]

# Create a DataFrame
dim_energy_contract_df = pd.DataFrame(energy_contracts)

In [ ]:
# Display the DataFrame for verification
print(dim_energy_contract_df)

In [ ]:
# Write to SQL Server
dim_energy_contract_df.to_sql('DimEnergyContract', con=engine, if_exists='append', index=False)

print(f"Successfully inserted {len(dim_energy_contract_df)} energy contracts into DimEnergyContract table.")

Voeg tariefkaarten toe:

In [ ]:
# Read the CSV file with semicolon delimiter
csv_file_path = '../data/input/energy_costs.csv'
df = pd.read_csv(csv_file_path, sep=';')

# Get provider and contract name from EnergyCostKey
df['Provider'] = df['EnergyCostKey'].apply(lambda x: x.split('_')[1])
df['ContractName'] = df['EnergyCostKey'].apply(lambda x: x.split('_')[2])

# Convert DateKey from 'YYYY-MM-DD' to integer format 'YYYYMMDD'
df['DateKey'] = df['DateKey'].apply(lambda x: int(x.replace('-', '')))

# Replace empty strings with None for proper SQL NULL values
df = df.replace('', None)

# Look up ContractKey from DimEnergyContract table
# First, get the existing contracts from the database
contracts_query = "SELECT ContractKey, Provider, ContractName FROM DimEnergyContract"
contracts_df = pd.read_sql(contracts_query, engine)

# Create a dictionary mapping (Provider, ContractName) to ContractKey
contract_keys = {}
for _, row in contracts_df.iterrows():
    contract_keys[(row['Provider'], row['ContractName'])] = row['ContractKey']

# Map the ContractKey to each row
df['ContractKey'] = df.apply(lambda row: contract_keys.get((row['Provider'], row['ContractName'])), axis=1)

# Drop the original EnergyCostKey and the temporary Provider and ContractName columns
df = df.drop(['EnergyCostKey', 'Provider', 'ContractName'], axis=1)


In [ ]:
# Display the first few rows to verify the transformation
print(df.head())

In [ ]:
 # Insert the data into the SQL Server database
df.to_sql('FactEnergyCost', con=engine, if_exists='append', index=False)

print(f"Successfully inserted {len(df)} rows into FactEnergyCost table.")